# RAG-Tutorial: Bauhaftpflicht-Fälle durchsuchen

Dieses Notebook erklärt Schritt für Schritt, wie unser **Retrieval-Augmented Generation (RAG)**-System funktioniert.

---

## 1. Was ist RAG?

RAG steht für **Retrieval-Augmented Generation** — ein Muster, bei dem ein LLM (Large Language Model) mit externem Wissen ergänzt wird.

```
Frage des Benutzers
        │
        ▼
┌───────────────┐
│   Retriever   │  ← Sucht relevante Dokumente in der Vektor-Datenbank
└───────┬───────┘
        │ Top-K Chunks
        ▼
┌───────────────┐
│     LLM       │  ← Generiert Antwort basierend auf Frage + Kontext
└───────┬───────┘
        │
        ▼
   Antwort mit Quellen
```

**Ohne RAG** kennt das LLM nur sein Trainings-Wissen — es weiss nichts über unsere spezifischen Versicherungsfälle.

**Mit RAG** füttern wir das LLM mit den relevantesten Dokumenten aus unserer Datenbank, bevor es antwortet.

## 2. Warum RAG?

LLMs haben drei fundamentale Probleme, die RAG löst:

| Problem | Ohne RAG | Mit RAG |
|---------|----------|--------|
| **Halluzinationen** | LLM erfindet Fakten | Antwort basiert auf echten Dokumenten |
| **Veraltetes Wissen** | Trainings-Cutoff | Zugriff auf aktuelle Daten |
| **Kein Domänenwissen** | Kennt unsere Fälle nicht | Durchsucht unsere Falldatenbank |

### Unser Anwendungsfall

Ein Senior Underwriter für Bauhaftpflicht möchte wissen:
- *"Gab es ähnliche Fälle mit undichten Duschen?"*
- *"Wie wurde in vergleichbaren Fällen die Haftung aufgeteilt?"*
- *"Welche Beträge wurden bei Wasserschäden gezahlt?"*

Das LLM allein kann diese Fragen nicht beantworten — es braucht unsere Falldaten.

## 3. Die Komponenten

### 3.1 Chunking

Dokumente sind oft zu lang, um sie komplett dem LLM zu geben. Deshalb teilen wir sie in **Chunks** (Abschnitte).

Wir verwenden den `RecursiveCharacterTextSplitter` von LangChain:
- **chunk_size=1000**: Jeder Chunk ist ca. 1000 Zeichen lang
- **chunk_overlap=200**: 200 Zeichen Überlappung, damit kein Kontext verloren geht
- **Rekursiv**: Versucht zuerst an Absätzen zu trennen, dann an Sätzen, dann an Wörtern

### 3.2 Embeddings

Jeder Chunk wird durch ein **Embedding-Modell** in einen Zahlenvektor umgewandelt. Ähnliche Texte haben ähnliche Vektoren.

Wir nutzen `text-embedding-3-small` von OpenAI.

### 3.3 Vector Database (ChromaDB)

Die Vektoren werden in **ChromaDB** gespeichert. Bei einer Suche wird die Frage ebenfalls in einen Vektor umgewandelt und die ähnlichsten Chunks werden zurückgegeben.

### 3.4 LLM (gpt-4o-mini)

Das LLM erhält die Frage zusammen mit den gefundenen Chunks und generiert eine strukturierte Antwort.

## 4. Design-Entscheidung: Metadaten NICHT in den Chunk

Eine wichtige Architektur-Entscheidung in diesem Projekt:

### Schlechter Ansatz (3_rag)

```
page_content = """
[Fall W1 | Wasser/Abdichtung – Duschen | CHF 95'000]
[Dokument: schadenanmeldung | 2024-02-20]
---
Wir erklären hiermit den Schadensfall...
"""
```

**Problem**: Der Header verfälscht die Embedding-Vektoren. Wenn jemand nach "Wasseraustritt" sucht, beeinflusst der Header-Text das Suchergebnis.

### Guter Ansatz (unser Projekt)

```python
page_content = "Wir erklären hiermit den Schadensfall..."
metadata = {
    "case_id": "W1",
    "cluster": "Wasser/Abdichtung – Duschen",
    "schaden_chf": 95000,
    ...
}
```

**Vorteil**: 
- Der Vektor repräsentiert nur den **echten Inhalt** des Dokuments
- Metadaten stehen im `metadata`-Dict und können zum **Filtern** verwendet werden
- Erst beim Prompt-Bau fügen wir den Metadaten-Header hinzu, damit das LLM die Quelle kennt

## 5. Code-Walkthrough

### `config.py` — Zentrale Konfiguration

Alle Pfade, Modellnamen und Parameter an einem Ort:

```python
DATA_PATH = BASE_DIR / ".." / "2_syntetic_data" / "output"  # Quelldaten
CHROMA_PATH = BASE_DIR / "chroma_db"                        # Vektor-DB
CHUNK_SIZE = 1000                                            # Chunk-Grösse
CHUNK_OVERLAP = 200                                          # Überlappung
EMBEDDING_MODEL = "text-embedding-3-small"                   # Embedding-Modell
LLM_MODEL = "gpt-4o-mini"                                   # LLM
```

### `indexer.py` — Dokumente laden und indexieren

1. Liest alle Fall-Ordner (`W1`, `W2`, `F1`, ...)
2. Lädt `case_bible.json` (Metadaten) und alle Dokument-JSONs
3. Erstellt LangChain `Document`-Objekte mit reinem Text + Metadaten
4. Chunked mit `RecursiveCharacterTextSplitter`
5. Speichert in ChromaDB

### `retriever.py` — Suche

Eine generische `search()`-Funktion mit optionalem `filter_dict`:

```python
# Ohne Filter — alle Fälle durchsuchen
search("Wasseraustritt", k=5)

# Mit Filter — nur Fall W1
search("Wasseraustritt", k=5, filter_dict={"case_id": "W1"})
```

### `rag_chain.py` — RAG-Logik

1. Ruft `search()` auf
2. Baut Prompt mit Metadaten-Header (zur Laufzeit, nicht im Vektor!)
3. Sendet an LLM mit System-Prompt
4. Dedupliziert Quellen

### `app.py` — Streamlit Web-UI

Bietet eine Web-Oberfläche mit:
- Eingabefeld für Fragen
- Sidebar-Filter für Fall-ID und Cluster
- Anzeige der Antwort und Quellen

## 6. Interaktive Demos

Jetzt wird's praktisch! Die folgenden Zellen kannst du direkt ausführen.

### 6.1 Indexierung durchführen

Zuerst müssen wir die Dokumente in die Vektor-Datenbank laden:

In [ ]:
from indexer import index_all_cases

count = index_all_cases()
print(f"\n==> {count} Chunks erfolgreich indexiert!")

### 6.2 Suche ausführen und Chunks inspizieren

Schauen wir uns an, was die Vektor-Suche zurückgibt:

In [ ]:
from retriever import search, get_collection_stats

# Statistiken anzeigen
stats = get_collection_stats()
print(f"Collection: {stats}\n")

# Suche durchführen
results = search("Wasseraustritt im Duschbereich", k=3)

for i, (doc, score) in enumerate(results):
    print(f"=== Ergebnis {i + 1} (Score: {score:.3f}) ===")
    print(f"Fall:    {doc.metadata.get('case_id')}")
    print(f"Cluster: {doc.metadata.get('cluster')}")
    print(f"Typ:     {doc.metadata.get('doc_typ')}")
    print(f"Datum:   {doc.metadata.get('doc_datum')}")
    print(f"CHF:     {doc.metadata.get('schaden_chf')}")
    print(f"Inhalt:  {doc.page_content[:300]}...")
    print()

### 6.3 Gefilterte Suche

Wir können die Suche auf bestimmte Fälle oder Cluster einschränken:

In [ ]:
# Nur in Fall W1 suchen
print("=== Suche nur in Fall W1 ===")
results_w1 = search("Abdichtung", k=3, filter_dict={"case_id": "W1"})
for doc, score in results_w1:
    print(f"  [{score:.3f}] {doc.metadata['doc_typ']} — {doc.page_content[:100]}...")

print()

# Nur Fälle mit bestimmtem Cluster
print("=== Suche im Cluster 'Wasser/Abdichtung – Duschen' ===")
results_cluster = search(
    "Haftung",
    k=3,
    filter_dict={"cluster": "Wasser/Abdichtung – Duschen"},
)
for doc, score in results_cluster:
    print(f"  [{score:.3f}] Fall {doc.metadata['case_id']}: {doc.metadata['doc_typ']}")

### 6.4 Vollständige RAG-Anfrage mit Quellenangabe

Jetzt das volle System: Suche + LLM-Antwort + Quellen:

In [ ]:
from rag_chain import ask

result = ask(
    "Gibt es Fälle mit Wasserabdichtungsproblemen? Wie wurden sie gelöst?",
    verbose=True,
)

print("\n" + "=" * 60)
print("ANTWORT:")
print("=" * 60)
print(result["answer"])

print("\nQUELLEN:")
for s in result["sources"]:
    print(f"  - Fall {s['case_id']}: {s['doc_typ']} ({s['doc_datum']}) — {s['cluster']}")

In [ ]:
# Noch ein Beispiel: Mit Filter auf einen bestimmten Fall
result = ask(
    "Was sind die wichtigsten Fakten zu diesem Fall?",
    filter_dict={"case_id": "W3"},
    verbose=True,
)

print("\n" + "=" * 60)
print("ANTWORT:")
print("=" * 60)
print(result["answer"])

## 7. Skalierung: Was passiert bei 400'000+ Dokumenten?

Unser bisheriges System funktioniert gut für 20 Fälle. Aber was, wenn wir es produktiv einsetzen?

### Die realen Zahlen

```
5 Jahre × 5'500 Fälle/Jahr × 15 Dokumente/Fall = ~412'500 Dokumente
                                                 = ~2–4 Millionen Chunks
```

Bei dieser Grössenordnung stossen wir auf **drei fundamentale Probleme**.

### Problem 1: Nicht jede Frage ist eine Ähnlichkeitssuche

Unser aktuelles System macht immer das Gleiche: Frage → Vektor-Suche → Top 5 Chunks → LLM. Aber in der Praxis gibt es sehr unterschiedliche Fragetypen:

| Fragetyp | Beispiel | Was eigentlich nötig wäre |
|-----------|---------|---------------------------|
| **Vergleichsfall** | "Gibt es ähnliche Fälle mit undichten Duschen?" | Klassische Vektor-Suche — unser System kann das |
| **Mitarbeiter-Analyse** | "Wie erledigt Herr Müller seine Wasserschaden-Fälle?" | Filtern nach Sachbearbeiter, dann *viele* Dokumente lesen und aggregieren |
| **Abteilungs-Übersicht** | "Wie viele offene Fälle hat Team Ost im Cluster Wasser?" | Strukturierte Abfrage auf Metadaten (SQL-artig), kein Embedding nötig |
| **Spezifischer Fall** | "Was ist der aktuelle Stand von Fall W3?" | Exakter Filter auf case_id, dann *alle* Dokumente dieses Falls laden |
| **Norm-Recherche** | "In welchen Fällen wurde OR 371 angewendet?" | Keyword-Suche nach exaktem Begriff, Vektor-Suche versagt hier oft |

**Top-K=5 löst nur den ersten Typ.** Für die restlichen brauchen wir andere Strategien.

### Problem 2: Nadel im Heuhaufen

Bei 3 Millionen Chunks liefert eine Vektor-Suche mit k=5 die **0.00017%** ähnlichsten Chunks. Die Wahrscheinlichkeit, relevante aber anders formulierte Treffer zu verpassen, ist sehr hoch. Besonders bei mehrsprachigen Dokumenten (DE/FR/IT) kann ein Treffer semantisch identisch, aber sprachlich völlig anders formuliert sein.

### Problem 3: LLM-Kontextfenster ist begrenzt

Selbst wenn du k=50 setzt, passen ca. 50'000 Zeichen in den Prompt. Bei analytischen Fragen ("Übersicht über alle Wasserschaden-Fälle der letzten 3 Jahre") bräuchtest du Hunderte von Dokumenten — das sprengt jedes Kontextfenster.

## 8. Lösung 1: ParentDocumentRetriever

### Das Grundproblem

Beim Chunking stehen wir vor einem Dilemma:
- **Kleine Chunks** (200 Zeichen) → präzisere Suche, aber zu wenig Kontext fürs LLM
- **Grosse Chunks** (2000 Zeichen) → genug Kontext fürs LLM, aber unpräzise Suche

### Die Lösung: Zwei Ebenen

Der ParentDocumentRetriever löst das elegant mit zwei Chunk-Grössen:

```
Originaldokument (z.B. 5'000 Zeichen)
    │
    ├── Parent-Chunk 1 (2000 Zeichen)  ← Wird ans LLM gegeben
    │       ├── Child-Chunk 1a (200 Zeichen)  ← Wird embedded + gesucht
    │       ├── Child-Chunk 1b (200 Zeichen)  ← Wird embedded + gesucht
    │       └── Child-Chunk 1c (200 Zeichen)  ← Wird embedded + gesucht
    │
    └── Parent-Chunk 2 (2000 Zeichen)  ← Wird ans LLM gegeben
            ├── Child-Chunk 2a (200 Zeichen)  ← Wird embedded + gesucht
            └── Child-Chunk 2b (200 Zeichen)  ← Wird embedded + gesucht
```

**Ablauf:**
1. Benutzer stellt Frage
2. Vektor-Suche findet den besten **Child-Chunk** (klein, präzise)
3. System holt den zugehörigen **Parent-Chunk** (gross, kontextreich)
4. Parent-Chunk wird ans LLM gegeben

### Wann sinnvoll?

- Bei langen Dokumenten (Expertisen, Klageantworten, Urteile)
- Wenn die Suchpräzision bei vielen Chunks leidet
- Gut kombinierbar mit allen anderen Lösungen unten

### Limitierung

Löst **nicht** das Problem der verschiedenen Fragetypen — es verbessert nur die Qualität der Vektor-Suche selbst.

## 9. Lösung 2: Query Routing

### Idee

Bevor wir überhaupt suchen, lassen wir ein LLM die Frage **klassifizieren** und wählen dann die passende Strategie:

```
Benutzerfrage
      │
      ▼
┌─────────────────┐
│  LLM-Classifier │  "Was für eine Frage ist das?"
└────────┬────────┘
         │
    ┌────┼──────────┬──────────────┐
    ▼    ▼          ▼              ▼
 Vektor  Metadata   Keyword    Alle Docs
 Suche   Filter     Suche      eines Falls
```

### Konkret

Ein kleiner Prompt klassifiziert die Frage:

```python
ROUTING_PROMPT = """Klassifiziere die Frage in eine Kategorie:
- SIMILARITY: Suche nach ähnlichen Fällen
- CASE_LOOKUP: Frage zu einem spezifischen Fall (enthält Fall-ID)
- AGGREGATION: Übersicht, Statistik, Zählung
- KEYWORD: Suche nach exaktem Begriff (Norm, Gesetz, Name)

Frage: {question}
Kategorie:"""
```

Je nach Ergebnis wird eine andere Suchstrategie verwendet:
- **SIMILARITY** → Vektor-Suche mit k=10 + Reranking
- **CASE_LOOKUP** → Metadata-Filter `{"case_id": "W3"}`, alle Dokumente laden
- **AGGREGATION** → SQL-Query auf Metadaten-Tabelle
- **KEYWORD** → BM25 Volltextsuche statt Embedding

### Vorteil

Einfach zu implementieren, ein LLM-Call für die Klassifikation, dann deterministische Logik.

### Limitierung

Funktioniert nur, wenn die Fragetypen klar trennbar sind. Bei komplexen Fragen, die *mehrere Strategien* brauchen ("Wie hat Herr Müller ähnliche Wasserschäden gelöst und wie viele waren es?"), stösst Routing an seine Grenzen.

## 10. Lösung 3: Agentic RAG

Das ist die mächtigste — aber auch komplexeste — Lösung. Hier wird das Konzept konkret erklärt.

### Was ist ein "Agent"?

Ein Agent ist ein LLM, das **nicht einfach antwortet**, sondern **selbst entscheidet, was es als Nächstes tun muss**. Es hat Zugang zu **Tools** (Werkzeugen) und ruft diese in einer Schleife auf, bis es genug Information hat, um zu antworten.

```
Benutzerfrage
      │
      ▼
┌──────────────────────────────────────────────────────┐
│  AGENT (LLM in einer Schleife)                       │
│                                                      │
│  Schritt 1: "Ich muss zuerst herausfinden, welche   │
│              Fälle Herr Müller bearbeitet hat."       │
│              → Ruft Tool: metadata_query auf          │
│                                                      │
│  Schritt 2: "OK, Müller hat 47 Fälle. Davon sind    │
│              12 im Cluster Wasser. Ich schaue mir     │
│              die Abschluss-Dokumente an."             │
│              → Ruft Tool: vector_search auf           │
│                                                      │
│  Schritt 3: "Die Chunks zeigen ein Muster. Ich       │
│              brauche noch die Vergleichsbeträge."     │
│              → Ruft Tool: metadata_query auf          │
│                                                      │
│  Schritt 4: "Jetzt habe ich genug Info."             │
│              → Generiert finale Antwort               │
└──────────────────────────────────────────────────────┘
```

### Die Tools des Agenten

Das Entscheidende bei Agentic RAG sind die **Tools**, die dem Agenten zur Verfügung stehen. Der Agent "sieht" für jedes Tool eine Beschreibung und entscheidet selbst, wann er welches Tool einsetzt.

#### Tool 1: `vector_search`
Semantische Ähnlichkeitssuche — das, was unser aktuelles System macht.

```python
@tool
def vector_search(query: str, k: int = 10, filter_dict: dict = None) -> str:
    """Durchsucht die Falldatenbank nach semantisch ähnlichen Dokumenten.
    
    Nutze dieses Tool, wenn du nach inhaltlich ähnlichen Fällen, 
    Argumentationen oder Sachverhalten suchst.
    
    Args:
        query: Die Suchanfrage in natürlicher Sprache
        k: Anzahl Ergebnisse (Standard: 10)
        filter_dict: Optionaler Filter, z.B. {"case_id": "W1"}
    """
    results = retriever.search(query, k=k, filter_dict=filter_dict)
    # Formatiert als lesbarer Text für den Agenten
    return format_results(results)
```

#### Tool 2: `metadata_query`
Strukturierte Abfrage auf den Metadaten — wie eine SQL-Query.

```python
@tool
def metadata_query(
    filter_dict: dict, 
    return_fields: list[str] = None,
    count_only: bool = False
) -> str:
    """Durchsucht die Metadaten der Fälle strukturiert.
    
    Nutze dieses Tool für Zählungen, Übersichten und Filterungen,
    wenn du NICHT nach Textinhalten suchst, sondern nach strukturierten
    Informationen wie: Anzahl Fälle, Liste von Fall-IDs, Beträge, Status.
    
    Args:
        filter_dict: Filter, z.B. {"sachbearbeiter": "Müller", "status": "offen"}
        return_fields: Welche Felder zurückgeben, z.B. ["case_id", "schaden_chf"]
        count_only: Wenn True, nur die Anzahl Treffer zurückgeben
    
    Beispiele:
        - Wie viele offene Fälle? → filter={"status": "offen"}, count_only=True
        - Alle Fälle von Müller? → filter={"sachbearbeiter": "Müller"}, 
                                    return_fields=["case_id", "cluster", "schaden_chf"]
    """
    # Abfrage auf Metadaten-Tabelle (z.B. PostgreSQL oder Pandas DataFrame)
    return query_metadata_store(filter_dict, return_fields, count_only)
```

#### Tool 3: `get_case_documents`
Lädt *alle* Dokumente eines bestimmten Falls.

```python
@tool
def get_case_documents(case_id: str, doc_typ: str = None) -> str:
    """Lädt alle Dokumente eines bestimmten Falls.
    
    Nutze dieses Tool, wenn du einen bestimmten Fall im Detail 
    analysieren musst — z.B. alle Dokumente von Fall W3 oder nur 
    das Urteil von Fall H2.
    
    Args:
        case_id: Die Fall-ID, z.B. "W3"
        doc_typ: Optional: Nur Dokumente dieses Typs, z.B. "urteilsauszug"
    """
    docs = load_all_documents_for_case(case_id, doc_typ)
    return format_documents(docs)
```

#### Tool 4: `summarize_text`
Fasst lange Texte zusammen — wichtig, wenn der Agent zu viel Text hat.

```python
@tool
def summarize_text(text: str, focus: str = None) -> str:
    """Fasst einen langen Text zusammen.
    
    Nutze dieses Tool, wenn du zu viel Text hast, um ihn direkt
    zu verarbeiten. Gib optional einen Fokus an, z.B. 
    "Konzentriere dich auf die Haftungsfrage".
    
    Args:
        text: Der zu zusammenfassende Text
        focus: Optionaler Fokus für die Zusammenfassung
    """
    return llm_summarize(text, focus)
```

### Konkretes Beispiel: Komplexe Frage

**Frage:** *"Wie erledigt Herr Müller seine Wasserschaden-Fälle typischerweise? Gibt es ein Muster?"*

So würde der Agent vorgehen:

```
Agent denkt: "Das ist eine analytische Frage über einen Mitarbeiter.
              Ich muss zuerst seine Fälle finden, dann die Wasserschäden
              filtern und dann die Abschluss-Dokumente analysieren."

→ Tool-Aufruf 1: metadata_query(
      filter_dict={"sachbearbeiter": "Müller", "cluster": {"$contains": "Wasser"}},
      return_fields=["case_id", "status", "schaden_chf"]
  )
← Ergebnis: "12 Fälle gefunden: W1 (vergleich, CHF 95'000), W4 (vergleich, CHF 42'000), ..."

Agent denkt: "OK, 12 Fälle, die meisten per Vergleich erledigt.
              Ich schaue mir die Vergleichsangebote und Abschlüsse genauer an."

→ Tool-Aufruf 2: vector_search(
      query="Vergleichsverhandlung Einigung Abschluss",
      k=15,
      filter_dict={"sachbearbeiter": "Müller", "cluster": {"$contains": "Wasser"}}
  )
← Ergebnis: 15 Chunks aus verschiedenen Abschluss-Dokumenten

Agent denkt: "Das ist viel Text. Ich fasse die Muster zusammen."

→ Tool-Aufruf 3: summarize_text(
      text=[die 15 Chunks],
      focus="Muster in der Vorgehensweise bei Vergleichsverhandlungen"
  )
← Ergebnis: "Zusammenfassung: Müller holt typischerweise zuerst ein 
             Parteigutachten ein, dann eine gemeinsame Expertise, und 
             schliesst meist mit einem Vergleich bei 40-60% der Forderung ab."

Agent denkt: "Jetzt habe ich genug Information für eine fundierte Antwort."

→ Generiert finale Antwort mit Quellen
```

### Der entscheidende Unterschied zu "normalem" RAG

| | Normales RAG | Agentic RAG |
|---|---|---|
| **Ablauf** | Fix: Suche → Prompt → Antwort | Dynamisch: Agent entscheidet bei jeder Frage neu |
| **Anzahl Suchen** | Immer genau 1 | So viele wie nötig (typisch 2–5) |
| **Suchstrategie** | Immer Vektor-Suche | Wählt aus: Vektor, Metadata, Keyword, Volldokument |
| **Umgang mit viel Text** | Alles in einen Prompt | Kann zwischendurch zusammenfassen |
| **Kosten** | 1 LLM-Call | 3–10 LLM-Calls pro Frage |
| **Latenz** | ~2 Sekunden | ~10–30 Sekunden |
| **Komplexität** | Einfach | Deutlich komplexer zu bauen und debuggen |

### Frameworks für Agentic RAG

- **LangGraph** (von LangChain) — Graph-basierte Agent-Logik, gut für kontrollierte Abläufe
- **LlamaIndex Agents** — Eingebaute RAG-Agenten mit Query Planning
- **CrewAI** — Multi-Agenten-System (z.B. ein "Recherche-Agent" und ein "Analyse-Agent")
- **Eigenbau mit OpenAI Function Calling** — Maximale Kontrolle, mehr Aufwand

## 11. Lösung 4: Hybrid Search + Reranking

### Hybrid Search

Kombination aus zwei Suchmethoden:

```
Frage: "Wurde OR 371 in Fällen mit Abdichtungsmängeln angewendet?"
      │
      ├── Vektor-Suche → Findet semantisch ähnliche Chunks
      │                   (gut für "Abdichtungsmängel")
      │
      ├── Keyword-Suche (BM25) → Findet exakte Begriffe
      │                           (gut für "OR 371")
      │
      ▼
  Ergebnisse verschmelzen (Reciprocal Rank Fusion)
      │
      ▼
  Kombinierte Top-K Ergebnisse
```

**Warum beides?** Vektor-Suche versteht Bedeutung ("Wasseraustritt" ≈ "infiltration d'eau"), aber versagt bei exakten Fachbegriffen. Keyword-Suche findet "OR 371" zuverlässig, versteht aber keine Synonyme.

### Reranking

Ein zweistufiger Prozess: zuerst breit suchen, dann präzise filtern.

```
Schritt 1: Vektor-Suche mit k=50  (schnell, aber ungenau)
                │
                ▼
Schritt 2: Cross-Encoder bewertet alle 50 Ergebnisse neu
           (langsam, aber viel präziser)
                │
                ▼
Schritt 3: Nur die besten 5–10 ans LLM
```

Ein **Cross-Encoder** (z.B. `cross-encoder/ms-marco-MiniLM-L-6-v2`) bewertet Frage+Chunk als *Paar* — das ist deutlich genauer als die Vektor-Ähnlichkeit, aber zu langsam für Millionen von Chunks. Deshalb die Zweistufigkeit.

### Wann sinnvoll?

- Hybrid Search: Immer, wenn exakte Begriffe wichtig sind (Normen, Gesetze, Namen, Fall-IDs)
- Reranking: Bei grossem Corpus, wo die Top-5 der Vektor-Suche oft nicht die besten Treffer sind

## 12. Lösung 5: Skalierbare Vector-Datenbank

ChromaDB ist hervorragend zum Lernen und für Prototypen. Bei 2–4 Millionen Chunks braucht man aber eine produktionsreife Lösung:

| Datenbank | Typ | Hybrid Search | Besonderheit |
|-----------|-----|:---:|---|
| **Qdrant** | Self-hosted oder Cloud | Ja | Sehr schnell, guter Filter-Support, Rust-basiert |
| **Weaviate** | Self-hosted oder Cloud | Ja | GraphQL API, eingebaute Hybrid Search |
| **Pinecone** | Nur Cloud (Managed) | Ja | Skaliert automatisch, kein Ops-Aufwand |
| **pgvector** | PostgreSQL-Extension | Nein* | Ideal wenn Postgres schon vorhanden, einfache Integration |
| **Milvus** | Self-hosted oder Cloud | Ja | Für sehr grosse Datenmengen (Milliarden Vektoren) |

*pgvector kann mit `pg_trgm` oder Volltextsuche kombiniert werden.

### Entscheidungshilfe

- **Ihr habt schon PostgreSQL** → pgvector (geringster Aufwand)
- **Maximale Kontrolle, self-hosted** → Qdrant (beste Performance/Preis)
- **Kein Ops-Team, einfach skalieren** → Pinecone (Managed Service)
- **Multi-Tenant, viele Teams** → Weaviate (gute Mandantentrennung)

## 13. Empfohlene Ausbaustufen

Nicht alles auf einmal bauen! Ein pragmatischer Stufenplan:

```
Stufe 0 (jetzt)     Stufe 1              Stufe 2              Stufe 3
─────────────────    ─────────────────    ─────────────────    ─────────────────
Einfaches RAG        + Hybrid Search      + Query Routing      + Agentic RAG
ChromaDB             + Reranking          + Metadata-Queries   + Multi-Tool Agent
Top-K=5              + Qdrant/pgvector    + ParentDoc-Retr.    + Zusammenfassung
                     + Grösseres K                             + Evaluation
                                                                 (RAGAS)
```

| Stufe | Löst | Aufwand | Wann umsetzen? |
|-------|------|---------|----------------|
| **0** | Grundfunktion: ähnliche Fälle finden | Fertig (dieses Projekt) | Jetzt |
| **1** | Bessere Trefferqualität bei vielen Daten | 1–2 Wochen | Sobald die echten Daten da sind |
| **2** | Verschiedene Fragetypen (Mitarbeiter, Übersicht, spez. Fall) | 2–3 Wochen | Wenn Benutzer unterschiedliche Fragen stellen |
| **3** | Komplexe, mehrstufige Analysen | 3–5 Wochen | Wenn die anderen Stufen nicht mehr reichen |

Jede Stufe baut auf der vorherigen auf — nichts muss weggeworfen werden.